# Artificial and Computational Intelligence

## Problem solving by Uninformed & Informed Search

Things to follow
1.	Use appropriate data structures to represent the graph and the path using python libraries
2.	Provide proper documentation
3.	Find the path and print it

Coding begins here

### 1.	Define the environment in the following block

List the PEAS decription of the problem here in this markdown block

Performance Measure:
- Winning the Game: The primary performance measure is winning the game by capturing all of the opponent's pieces.
- Minimizing Moves: Secondary performance measure could be minimizing the number of moves to win the game.
- Avoiding Loss: Ensuring that the agent does not lose the game by protecting its own pieces.

Environment:
- Game Board: A 3x3 grid representing the game state.
- Pieces: Different types of pieces (e.g., Soldier, Horse, Elephant) for both players.
- Turns: Alternating turns between the user and the AI.
- Rules: Movement rules for each type of piece and conditions for winning or losing the game.

Actuators:
- Move Pieces: The ability to move pieces on the board according to the rules of the game.
- Capture Pieces: The ability to capture opponent's pieces by moving to their position.

Sensors:
- Board State: The current configuration of pieces on the board.
- User Input: The moves entered by the user.
- Game Status: Information about whether the game is ongoing, won, lost or draw.

Design the agent as PSA Agent(Problem Solving Agent) 
Clear Initial data structures to define the graph and variable declarations is expected.
IMPORTANT: Write distinct code block as below

In [1]:
# Code Block : Set Initial State (Must handle dynamic inputs)

# Assumption: White pieces are represented by Capital letters and black pieces are represented by small letters
# Initial State
initial_state = [
    ['S', 'H', 'E'],
    ['', '', ''],
    ['', 's', 'h'],
]

In [2]:
# Code Block : Write function to design the Transition Model/Successor function. Ideally this would be called while search algorithms are implemented

def elephant_move(state, from_position, to_position):
    from_row, from_col = from_position
    to_row, to_col = to_position
    
    # From and to positions shouldn't be the same
    if from_row != to_row and from_col != to_col:
        return False
    
    # Piece cannot attack its own kind
    if state[from_row][from_col].isupper() and state[to_row][to_col].isupper():
        return False

    # Piece cannot attack its own kind
    if state[from_row][from_col].islower() and state[to_row][to_col].islower():
        return False

    # Vertical move
    if from_row == to_row:
        step = 1 if to_col > from_col else -1
        for col in range(from_col + step, to_col, step):
            if state[from_row][col] != '':
                return False
    # Horizontal move
    elif from_col == to_col:
        step = 1 if to_row > from_row else -1
        for row in range(from_row + step, to_row, step):
            if state[row][from_col] != '':
                return False
    
    # Update the state
    state[to_row][to_col] = state[from_row][from_col]
    state[from_row][from_col] = ''
    
    return True

def horse_move(state, from_position, to_position):
    from_row, from_col = from_position
    to_row, to_col = to_position
    
    # Absolute row and column difference
    row_diff = abs(to_row - from_row)
    col_diff = abs(to_col - from_col)

    # row and column differences between from and to positions should be 1 and 2 respectively.
    if (row_diff == 2 and col_diff == 1) or (row_diff == 1 and col_diff == 2):
        # Position should be within the board
        if 0 <= to_row < len(state) and 0 <= to_col < len(state[0]):
            from_piece = state[from_row][from_col]
            to_piece = state[to_row][to_col]
            # Update the state if to_position is empty or has an opponent piece
            if to_piece == '' or (from_piece.islower() != to_piece.islower()):
                state[to_row][to_col] = from_piece
                state[from_row][from_col] = ''
                return True
    return False

def soldier_move(state, from_position, to_position):
    from_row, from_col = from_position
    to_row, to_col = to_position
    
    # Get the direction based on the piece type
    direction = -1 if state[from_row][from_col] == 's' else 1
    
    # Valid step forward move for soldier
    if to_row == from_row + direction and to_col == from_col and state[to_row][to_col] == '':
        state[to_row][to_col] = state[from_row][from_col]
        state[from_row][from_col] = ''
        return True
    
    # Attack move for soldier
    if to_row == from_row + direction and abs(to_col - from_col) == 1:
        if state[to_row][to_col] != '' and state[to_row][to_col].islower() != state[from_row][from_col].islower():
            state[to_row][to_col] = state[from_row][from_col]
            state[from_row][from_col] = ''
            return True
    
    return False

def actions(state, player):
    legal_moves = []
    rows = len(state)
    cols = len(state[0])
    
    for row in range(rows):
        for col in range(cols):
            # Skip empty cells
            if state[row][col] == '':
                continue
            piece = state[row][col]
            from_position = (row, col)
            
            # Skip opponent pieces
            if (player == 'white' and piece.islower()) or (player == 'black' and piece.isupper()):
                continue
            
            # Generate legal moves for Soldier
            if piece in ['S', 's']:
                direction = -1 if piece == 's' else 1
                to_position = (row + direction, col)
                if 0 <= to_position[0] < rows and state[to_position[0]][to_position[1]] == '':
                    legal_moves.append((from_position, to_position))
                for dc in [-1, 1]:
                    to_position = (row + direction, col + dc)
                    if 0 <= to_position[0] < rows and 0 <= to_position[1] < cols:
                        if state[to_position[0]][to_position[1]] != '' and state[to_position[0]][to_position[1]].islower() != piece.islower():
                            legal_moves.append((from_position, to_position))
            
            # Generate legal moves for Horse
            elif piece in ['H', 'h']:
                possible_moves = [
                    (row + 2, col + 1), (row + 2, col - 1),
                    (row - 2, col + 1), (row - 2, col - 1),
                    (row + 1, col + 2), (row + 1, col - 2),
                    (row - 1, col + 2), (row - 1, col - 2)
                ]
                for to_position in possible_moves:
                    if 0 <= to_position[0] < rows and 0 <= to_position[1] < cols:
                        if state[to_position[0]][to_position[1]] == '' or state[to_position[0]][to_position[1]].islower() != piece.islower():
                            legal_moves.append((from_position, to_position))
            
            # Generate legal moves for Elephant
            elif piece in ['E', 'e']:
                for dc in [-1, 1]:
                    for step in range(1, cols):
                        to_position = (row, col + step * dc)
                        if 0 <= to_position[1] < cols:
                            if state[to_position[0]][to_position[1]] == '':
                                legal_moves.append((from_position, to_position))
                            elif state[to_position[0]][to_position[1]].islower() != piece.islower():
                                legal_moves.append((from_position, to_position))
                                break
                            else:
                                break
                for dr in [-1, 1]:
                    for step in range(1, rows):
                        to_position = (row + step * dr, col)
                        if 0 <= to_position[0] < rows:
                            if state[to_position[0]][to_position[1]] == '':
                                legal_moves.append((from_position, to_position))
                            elif state[to_position[0]][to_position[1]].islower() != piece.islower():
                                legal_moves.append((from_position, to_position))
                                break
                            else:
                                break
    return legal_moves

def result(state, move, player):
    from_position, to_position = move
    from_row, from_col = from_position

    # Copy the state to new_state    
    new_state = [row[:] for row in state]
    
    piece = state[from_row][from_col]

    # Validation checks
    if player == 'white' and piece.islower():
        raise ValueError("Invalid move: White player cannot move black pieces")
    if player == 'black' and piece.isupper():
        raise ValueError("Invalid move: Black player cannot move white pieces")

    # Validation check for available pieces
    if piece in ['S', 's']:
        if not soldier_move(new_state, from_position, to_position):
            raise ValueError("Invalid move for Soldier")
    elif piece in ['H', 'h']:
        if not horse_move(new_state, from_position, to_position):
            raise ValueError("Invalid move for Horse")
    elif piece in ['E', 'e']:
        if not elephant_move(new_state, from_position, to_position):
            raise ValueError("Invalid move for Elephant")
    else:
        raise ValueError("Unknown piece type")
    
    return new_state

In [3]:
# Code block : Write function to handle goal test (Must handle dynamic inputs). Ideally this would be called while search algorithms are implemented

# Default first player (cuurent player) is white
current_player_color = 'white'

def terminal_state(state):
    # Terminate if any player has no standing pieces or no legal moves for the current player
    white_coin_standing = False
    black_coin_standing = False

    for row in state:
        for coin in row:
            if coin in ['S', 'H', 'E']:
                white_coin_standing = True
            elif coin in ['s', 'h', 'e']:
                black_coin_standing = True

    return not white_coin_standing or not black_coin_standing or not actions(state, current_player_color)

### 2.	Definition of Algorithm (MINMAX Algorithm)

In [4]:
# Code Block : Function for algorithm implementation

def static_evaluation(state):
    # Assign piece weightage for static evaluation
    piece_values = {
        'S': 1, 's': -1,
        'H': 3, 'h': -3,
        'E': 5, 'e': -5,
    }
    # Calculate the score based on the piece weightage
    score = 0
    for row in state:
        for piece in row:
            if piece in piece_values:
                score += piece_values[piece]
    return score

def utility(state, player):
    # Terminal state check
    if not terminal_state(state):
        raise ValueError("The game is not in a terminal state.")
    
    white_coin_standing = False
    black_coin_standing = False

    # Check for standing pieces
    for row in state:
        for coin in row:
            if coin in ['S', 'H', 'E']:
                white_coin_standing = True
            elif coin in ['s', 'h', 'e']:
                black_coin_standing = True

    if player == 'white':
        if not black_coin_standing:
            return 1  # White wins
        elif not white_coin_standing:
            return 0  # White loses
    elif player == 'black':
        if not white_coin_standing:
            return 1  # Black wins
        elif not black_coin_standing:
            return 0  # Black loses
    
    return 0.5  # Draw (if applicable, adjust based on game rules)

# Initialize the max branching factor
max_branching_factor = 0

def minmax_algorithm(state, depth, is_maximizing_player):
    global max_branching_factor

    current_player = 'white' if is_maximizing_player else 'black'

    # If there are no possible actions, treat it as a terminal state
    if terminal_state(state) or depth == 0 or not actions(state, current_player):
        return static_evaluation(state)

    if is_maximizing_player:
        max_eval = float('-inf')
        for move in actions(state, 'white'):
            # Initialize branching factor
            branching_factor = 0
            new_state = result(state, move, 'white')
            eval = minmax_algorithm(new_state, depth - 1, False)
            max_eval = max(max_eval, eval)
            branching_factor += 1
        # Update the max_branching_factor if the current branching factor is greater
        if branching_factor > max_branching_factor:
            max_branching_factor = branching_factor
        return max_eval
    else:
        min_eval = float('inf')
        # Initialize branching factor
        branching_factor = 0
        # Invoke Minmax algorithm recursively if there are possible actions
        for move in actions(state, 'black'):
            new_state = result(state, move, 'black')
            eval = minmax_algorithm(new_state, depth - 1, True)
            min_eval = min(min_eval, eval)
            branching_factor += 1
        # Update the max_branching_factor if the current branching factor is greater
        if branching_factor > max_branching_factor:
            max_branching_factor = branching_factor
        return min_eval

def get_best_move(state, depth, player):
    # Get best move for AI player
    best_move = None
    best_value = float('-inf') if player == 'white' else float('inf')
    
    # Invoke Minmax algorithm recursively if there are possible actions
    for move in actions(state, player):
        new_state = result(state, move, player)
        move_value = minmax_algorithm(new_state, depth - 1, player == 'black')
        # Update the best move based on the player
        if (player == 'white' and move_value > best_value) or (player == 'black' and move_value < best_value):
            best_value = move_value
            best_move = move
    
    return best_move


### DYNAMIC INPUT

IMPORTANT : Dynamic Input must be got in this section. Display the possible states to choose from:
This is applicable for all the relevent problems as mentioned in the question. 

In [5]:
# Code Block : Function & call to get inputs (start/end state)

# Functions to get the inputs from the user is defined here. This will be invoked from main function.
def print_board(state):
    cols = len(state[0])
    
    # Print column headers
    print('    ' + '   '.join(map(str, range(cols))))
    print('  ' + '-' * (cols * 4 + 1))
    
    for row_idx, row in enumerate(state):
        # Print row index and row content
        print(f'{row_idx} | ' + ' | '.join(cell if cell != '' else ' ' for cell in row) + ' |')
        print('  ' + '-' * (cols * 4 + 1))

def get_user_move():
    from_position = input("Enter the from position: (e.g. x,y): ")
    to_position = input("Enter the to position (e.g. x,y): ")

    # Parse the user input and get row and column values for from and to positions
    from_row, from_col = map(int, from_position.split(','))
    to_row, to_col = map(int, to_position.split(','))

    return (from_row, from_col), (to_row, to_col)

def user_turn(current_state, user_color):
    valid = True
    retry = 0
    while valid:
        # Restricting the user to 3 retries
        if retry == 3:
            print("Max retries reached. Exiting game")
            return None
        try:
            print("User's Turn:")
            user_move = get_user_move()
            current_state = result(current_state, user_move, user_color)
            valid = False
        except Exception as err_message:
            print("Error:", err_message)
            print("Try again, you have", 3 - retry, "retries left")
            retry += 1
    return current_state

### 4.	Calling the search algorithms

In [6]:
# Invoke Minmax algorithm (Should Print the solution, path, cost etc., (As mentioned in the problem))

# Since the minmax algorithm might be called multiple times for gaming problem, we have used a function and have set gaming tree depth as 3.
def main():
    global current_player_color
    current_state = initial_state
    print("Initial Board:")
    print_board(current_state)

    user_color = input("Choose your color (white/black): ").strip().lower()
    if user_color not in ['white', 'black']:
        print("Invalid color. Defaulting to white")
        user_color = 'white'
    
    ai_color = 'black' if user_color == 'white' else 'white'

    print("User color selected:", user_color)
    print("AI color selected:", ai_color)

    while not terminal_state(current_state):
        if user_color == 'white':
            # User turn
            current_player_color = user_color
            current_state = user_turn(current_state, 'white')
            # Max retries reached
            if current_state is None:
                return

            print("Board after User's move:")
            print_board(current_state)
            
            # AI turn
            current_player_color = ai_color

            # Check if the game is over
            if terminal_state(current_state):
                break
            
            print("AI's Turn:")
            ai_move = get_best_move(current_state, 3, 'black')
            current_state = result(current_state, ai_move, 'black')
            print("Board after AI's move:")
            print_board(current_state)

            current_player_color = user_color
        else:
            # AI turn
            current_player_color = ai_color
            print("AI's Turn:")
            ai_move = get_best_move(current_state, 3, 'white')
            current_state = result(current_state, ai_move, 'white')
            print("Board after AI's move:")
            print_board(current_state)
            
            current_player_color = user_color

            # Check if the game is over
            if terminal_state(current_state):
                break
            
            # User turn
            current_state = user_turn(current_state, 'black')
            # Max retries reached
            if current_state is None:
                return

            print("Board after User's move:")
            print_board(current_state)

            current_player_color = ai_color
    
    print("Game Over")

    # Calculate the utility value and announce the winner
    if utility(current_state, 'white') == 1:
        print("White wins!")
    elif utility(current_state, 'black') == 1:
        print("Black wins!")
    else:
        print("It's a draw!")

if __name__ == "__main__":
    main()

Initial Board:
    0   1   2
  -------------
0 | S | H | E |
  -------------
1 |   |   |   |
  -------------
2 |   | s | h |
  -------------


Choose your color (white/black):  white


User color selected: white
AI color selected: black
User's Turn:


Enter the from position: (e.g. x,y):  0,1
Enter the to position (e.g. x,y):  2,2


Board after User's move:
    0   1   2
  -------------
0 | S |   | E |
  -------------
1 |   |   |   |
  -------------
2 |   | s | H |
  -------------
AI's Turn:
Board after AI's move:
    0   1   2
  -------------
0 | S |   | E |
  -------------
1 |   | s |   |
  -------------
2 |   |   | H |
  -------------
User's Turn:


Enter the from position: (e.g. x,y):  0,0
Enter the to position (e.g. x,y):  1,1


Board after User's move:
    0   1   2
  -------------
0 |   |   | E |
  -------------
1 |   | S |   |
  -------------
2 |   |   | H |
  -------------
Game Over
White wins!


### 5.	Comparitive Analysis

In [7]:
# Code Block : Print the Time & Space complexity of algorithm

print("Max Branching Factor:", max_branching_factor)
print("Max Depth:", 3)
print("Time Complexity: O(b^d) = ", max_branching_factor ** 3)
print("Space Complexity: O(d) = 3")

Max Branching Factor: 3
Max Depth: 3
Time Complexity: O(b^d) =  27
Space Complexity: O(d) = 3


### 6.	Provide your comparitive analysis or findings in no more than 3 lines in below section
Findings:
The Minimax algorithm has a time complexity of (O(b^d)) and a space complexity of (O(d)), making it computationally expensive for large game trees. Optimizations like alpha-beta pruning can help reduce the effective branching factor, improving performance.